# Demo (Tweepy V1.1 API)

### Setup environment

In [ ]:
import tweepy
from dotenv import load_dotenv
import os

load_dotenv() # Load .env file (see .env example)

consumer_key = os.getenv('API_KEY')
consumer_secret = os.getenv('API_KEY_SECRET')
access_token = os.getenv('ACCESS_TOKEN')
access_token_secret = os.getenv('ACCESS_TOKEN_SECRET')

auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret, access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

### Connect to MongoDB by pymongo

In [ ]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)

db = client.worldcup # name dataset "worldcup"

## Extract

### Make query

In [ ]:
search_query = "#JPN #HRV" # AND relationship 
no_of_tweets = 150

tweets = api.search_tweets(q=search_query, 
                           tweet_mode='extended', # to get text which over 140 words limitation
                           count=no_of_tweets)

## Load

### Insert data to MongoDB

In [ ]:
for tweet in tweets:
    try:
        content = {
            "User": tweet.user.name,
            "Location": tweet.user.location,
            "Date_Created": tweet.created_at,
            "Number_of_Likes": tweet.favorite_count,
            "Source_of_Tweet": tweet.source,
            "Tweet": tweet.full_text
        }
        db.jh.insert_one(content) # create table named "jh" and insert the content
    except ValueError as e:
        print(e)

## Connect to Google Big Query

### Setup environment

In [ ]:
from google.cloud import bigquery

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= os.getenv('GOOGLE_APPLICATION_CREDENTIALS')

bq_client = bigquery.Client()

jpn_vs_hrv = bq_client.get_table('tecky-capstone-project.worldcup.jpn_hrv') # Need to create a table "jpn_hrv" in BigQuery first, now use get_table to locate the table

### Get data from MongoDB, modify and then insert to BigQuery

In [ ]:
jh = db.jh.find() # Use Pymongo access collection "jh"
for row in jh:
    row['_id'] = str(row['_id']) # "_id" is default obeject and not json serializable, need to stringify it first
    row['Date_Created'] = str(row['Date_Created']) # "Date_Created" is default datetime format, need to change it (should use datetime format instead)
    bq_client.insert_rows_json(jpn_vs_hrv,[row]) # insert to BQ